# Mapping Modelling Results

#### Author: alicefeng

This notebook provides a way to quickly visualize model-generated crash risk scores on a map.  Risk scores are color coded from yellow (lowest) to red (highest).

Inputs: 
- inter_and_non_int.shp - a shapefile that combines intersection and non-intersection segments with their ids as generated in create_segments.py
- a csv file that contains risk score predictions, where each row corresponds to one segment

Output: 
- a HTML file of a Leaflet map with risk scores by segment plotted on it

In [1]:
#Import the necessary Python modules
import pandas as pd
import geopandas as gpd
import folium
import branca.colormap as cm
#from folium.plugins import MarkerCluster
#from folium.element import IFrame

In [2]:
# Update below lines as needed to point to where you have stored your model output
output_fp = '../data/processed/'
output_filename = 'seg_with_risk_score_adj.csv'

In [3]:
# Read in model results
output = pd.read_csv(output_fp + output_filename)
output['id'] = output['segment_id'].astype('str')

In [4]:
# Read in shapefile as a GeoDataframe
streets = gpd.read_file('../data/processed/maps/inter_and_non_int.shp')

# Set the projection as EPSG:3857 since the shapefile didn't export with one
streets.crs = {'init': 'epsg:3857'}

# Then reproject to EPSG:4326 to match what Leaflet uses
streets = streets.to_crs({'init': 'epsg:4326'})

In [5]:
# Merge on model results to the GeoDataframe
streets_w_risk = streets.merge(output, on='id')

In [6]:
streets_w_risk[streets_w_risk.risk_score>0]

,geometry,id,segment_id,target,pre_week,pre_month,pre_quarter,avg_week,AADT,SPEEDLIMIT,Struct_Cnd,Surface_Tp,F_F_Class,target_adj,pre_week_adj,pre_month_adj,pre_quarter_adj,risk_score
1,(LINESTRING (-71.12493097319626 42.23514052752...,20,20,0,0.0,0.0,0.0,0.020408,18330,30,2,6,4,0.0,0.0,1.0,2.0,0.042969
5,(LINESTRING (-71.12484184732081 42.23554933458...,46,46,0,0.0,0.0,1.0,0.020408,9967,30,2,6,4,0.0,0.0,0.0,0.0,0.001953
13,(LINESTRING (-71.12243549173725 42.24467101882...,110,110,0,0.0,0.0,0.0,0.020408,16769,1,2,6,3,0.0,1.0,1.0,2.0,0.035156
16,(LINESTRING (-71.12573305420399 42.25109076408...,177,177,0,0.0,0.0,0.0,0.020408,0,30,3,6,7,0.0,0.0,0.0,1.0,0.438086
24,(LINESTRING (-71.14355299195152 42.25469907411...,265,265,0,0.0,0.0,1.0,0.061224,9809,30,2,6,4,0.0,0.0,0.0,1.0,0.007812
26,(LINESTRING (-71.12428153689001 42.25489283233...,276,276,0,0.0,1.0,1.0,0.040816,0,30,3,6,7,0.0,0.0,0.0,1.0,0.019531
28,(LINESTRING (-71.12008941869451 42.25474800515...,293,293,0,0.0,0.0,0.0,0.020408,10038,25,2,6,7,0.0,0.0,0.0,0.0,0.011719
35,(LINESTRING (-71.14011221360816 42.25835998144...,346,346,0,0.0,3.0,4.0,0.081633,9809,30,2,6,7,0.0,0.0,1.0,1.0,0.062500
37,(LINESTRING (-71.11638978719877 42.25989973954...,370,370,1,0.0,0.0,0.0,0.000000,3617,20,2,6,5,0.0,0.0,0.0,0.0,0.617188
42,(LINESTRING (-71.13553619970483 42.25807095392...,392,392,0,0.0,1.0,1.0,0.040816,0,30,3,6,7,0.0,0.0,3.0,4.0,0.027344


In [11]:
# Make map

# First create basemap
boston_map = folium.Map([42.3601, -71.0589], tiles='Cartodb Positron', zoom_start=12)  #"Cartodb dark_matter" also nice

# Create style function to color segments based on their risk score
color_scale = cm.linear.YlOrRd.scale(0, 1)
    
# Then add on GeoDataframe of risk scores
folium.GeoJson(streets_w_risk[streets_w_risk.risk_score>0],  # filter dataframe to only seg with risk>0 to reduce size
               name='Benchmark Model',
               style_function=lambda feature: {
                  'color': color_scale(feature['properties']['risk_score'])
              }).add_to(boston_map)

# Add control to toggle between model layers
folium.LayerControl(position='bottomright').add_to(boston_map)

# Finally, add legend
color_scale.caption = "Risk Score"
boston_map.add_child(color_scale)

# Save map as separate html file
#boston_map.save('risk_map.html')

boston_map